# ECE 763 - Computer Vision - Project 03
## Babysitting the training of a Network

<p>Import libraries</p>
Keras Sequential API was used to build the network. The CIFAR datasets was downloaded from keras datasets. 

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPool2D, Dropout, Reshape, BatchNormalization
from tensorflow.keras import optimizers, regularizers
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm

<p>Import and preprocess the data. 
    Preprocessing scheme was standard scaling by subtracting mean and dividing by the standard deviation. </p>

In [ ]:
from tensorflow.keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
#conv to float
(X_train, y_train), (X_test, y_test)=(X_train.astype('float64'), y_train), (X_test.astype('float64'), y_test)
#class_names for cifar
class_name = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']
preprocess_flag=False

In [ ]:
def preprocess_data(X_train, X_test):
    X_train-=np.mean(X_train,axis=0)
    X_test-=np.mean(X_test,axis=0)
    X_train/=np.std(X_train,axis=0)
    X_test/=np.std(X_test,axis=0)
    return X_train, X_test
if preprocess_flag==False:
    X_train, X_test=preprocess_data(X_train, X_test)

In [ ]:
#shape of the image data. 
iRows,iCols=X_train[0].shape[0],X_train[0].shape[1]

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train)

### Build the CNN 
<p> A basic 3 layer convolutional network is built and babysitting for it's training is done. The model consists of 3 convolutional blocks with relu activations followed by max pooling and then followed by a FCN. </p> 

In [ ]:
#model hyperparam
LEARNING_RATE=1e-4
EPOCHS=10
L2_REG=0
BATCH_SIZE=128
DROPOUT=0.35

In [ ]:
def create_model(L2_REG, drop=False, bnorm=False):
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(3,3), padding= 'same', activation='relu',input_shape = (iRows, iCols,3)))
    model.add(MaxPool2D(pool_size=(2,2), strides = 2, padding='same'))

    model.add(Conv2D(filters=64, kernel_size=(3,3), padding= 'same', activation='relu',kernel_regularizer=regularizers.l2(L2_REG), bias_regularizer=regularizers.l2(L2_REG)))
    model.add(MaxPool2D(pool_size=(2,2), strides = 2, padding='same'))
    if drop==True:
        model.add(Dropout(DROPOUT))
    if bnorm==True:
        model.add(BatchNormalization())

    model.add(Conv2D(filters=64, kernel_size=(3,3), padding= 'same', activation='relu', kernel_regularizer=regularizers.l2(L2_REG), bias_regularizer=regularizers.l2(L2_REG)))
    model.add(MaxPool2D(pool_size=(2,2), strides = 2, padding='same'))
    if drop==True:
        model.add(Dropout(DROPOUT))
    if bnorm==True:
        model.add(BatchNormalization())


    model.add(Flatten())
    model.add(Dense(units = 512, activation='relu', kernel_regularizer=regularizers.l2(L2_REG), bias_regularizer=regularizers.l2(L2_REG)))
    model.add(Dense(units=10,activation='softmax'))
    return model 

In [ ]:
model=create_model(L2_REG)
model.summary()

In [ ]:
model.compile(optimizer=optimizers.SGD(learning_rate=LEARNING_RATE),loss='sparse_categorical_crossentropy',metrics=['sparse_categorical_accuracy'])

In [ ]:
model.evaluate(X_train,y_train)

In [ ]:
model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=1, verbose=1)

In [ ]:
#model hyperparam
LEARNING_RATE=1e-4
EPOCHS=10
L2_REG=1e3
BATCH_SIZE=128
DROPOUT=0.35

model=create_model(L2_REG)
model.compile(optimizer=optimizers.SGD(learning_rate=LEARNING_RATE),loss='sparse_categorical_crossentropy',metrics=['sparse_categorical_accuracy'])
model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=1, verbose=1, validation_data=(X_val,y_val))

In [ ]:
#model hyperparam
LEARNING_RATE=1e-3
EPOCHS=10
L2_REG=0
BATCH_SIZE=128
DROPOUT=0.35

X_tiny=X_train[:20]
y_tiny=y_train[:20]


In [ ]:
model=create_model(L2_REG)
model.compile(optimizer=optimizers.SGD(learning_rate=LEARNING_RATE),loss='sparse_categorical_crossentropy',metrics=['sparse_categorical_accuracy'])
history = model.fit(X_tiny,y_tiny,batch_size=32,epochs=50, verbose=1)

In [ ]:
def plot_curves(history):
    plt.plot(history.history['sparse_categorical_accuracy'])
    plt.plot(history.history['val_sparse_categorical_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'], loc='upper left')
    plt.show()

    # Plot training & validation loss values
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'], loc='upper left')
    plt.show()

In [ ]:
LEARNING_RATE=1e-6
EPOCHS=10
L2_REG=1e-6
BATCH_SIZE=64
DROPOUT=0.35

model=create_model(L2_REG)
model.compile(optimizer=optimizers.SGD(learning_rate=LEARNING_RATE),loss='sparse_categorical_crossentropy',metrics=['sparse_categorical_accuracy'])
model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=1,  validation_data=(X_val,y_val))

In [ ]:
LEARNING_RATE=1e5
EPOCHS=10
L2_REG=1e-6
BATCH_SIZE=64
DROPOUT=0.35

model=create_model(L2_REG)
model.compile(optimizer=optimizers.SGD(learning_rate=LEARNING_RATE),loss='sparse_categorical_crossentropy',metrics=['sparse_categorical_accuracy'])
model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=1,  validation_data=(X_val,y_val))

In [ ]:
LEARNING_RATE=0.6
EPOCHS=10
L2_REG=1e-6
BATCH_SIZE=64
DROPOUT=0.35

model=create_model(L2_REG)
model.compile(optimizer=optimizers.SGD(learning_rate=LEARNING_RATE),loss='sparse_categorical_crossentropy',metrics=['sparse_categorical_accuracy'])
model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=1,  validation_data=(X_val,y_val))

In [ ]:
val_acc=[]
lrs=[]
l2_regs=[]
for i in tqdm(range(100)):
    lr = 10**np.random.uniform(-3,-6)
    l2_reg = 10**np.random.uniform(-5, 1)
    model = create_model(l2_reg, 0.35)
    model.compile(optimizer=optimizers.SGD(learning_rate=lr),loss='sparse_categorical_crossentropy',metrics=['sparse_categorical_accuracy'])
    history = model.fit(X_train,y_train,batch_size=BATCH_SIZE,epochs=5, verbose=1,validation_data=(X_val,y_val))
    val_acc.append(history.history['val_sparse_categorical_accuracy'][-1])
    lrs.append(lr)
    l2_regs.append(l2_reg)
    print(history.history['val_sparse_categorical_accuracy'][-1], lr, l2_reg)

In [ ]:
logs=[val_acc,lrs, l2_regs]
logs=np.array(logs)
# logs=np.sort(logs,axis=0)

In [ ]:
top10=np.argsort(logs[0,:])[::-1][:10]
top10

In [ ]:
logs[:,top10]

In [ ]:
val_acc=[]
lrs=[]
l2_regs=[]
for i in tqdm(range(50)):
    lr = 10**np.random.uniform(-3,-4)
    l2_reg = 10**np.random.uniform(-4, -1)
    model = create_model(l2_reg,0.35)
    model.compile(optimizer=optimizers.SGD(learning_rate=lr),loss='sparse_categorical_crossentropy',metrics=['sparse_categorical_accuracy'])
    history = model.fit(X_train,y_train,batch_size=8,epochs=15, verbose=1,validation_data=(X_val,y_val))
    val_acc.append(history.history['val_sparse_categorical_accuracy'][-1])
    lrs.append(lr)
    l2_regs.append(l2_reg)
    print(history.history['val_sparse_categorical_accuracy'][-1], lr, l2_reg)

In [ ]:
logs=[val_acc,lrs, l2_regs]
logs=np.array(logs)
# logs=np.sort(logs,axis=0)

In [ ]:
top4=np.argsort(logs[0,:])[::-1][:4]
top4

In [ ]:
logs[:,top4]

In [ ]:
lr, l2_reg = logs[1,top4[0]],logs[2,top4[0]]

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
model = create_model(l2_reg,0.35)
model.compile(optimizer=optimizers.SGD(learning_rate=lr),loss='sparse_categorical_crossentropy',metrics=['sparse_categorical_accuracy'])
history = model.fit(X_train,y_train,batch_size=8,epochs=50, verbose=1,validation_data=(X_val,y_val), callbacks=[es])

In [ ]:
plot_curves(history)

In [ ]:
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
y_pred = model.predict_classes(X_test)
mat = confusion_matrix(y_test,y_pred)
plot_confusion_matrix(mat,figsize=(9,9), show_normed=True)